In [1]:
import numpy as np
import networkx as nx
from functools import partial

from qokit.qaoa_circuit_maxcut import get_qaoa_circuit
from qokit.maxcut import maxcut_obj, get_adjacency_matrix
from qokit.utils import brute_force, objective_from_counts, invert_counts
from qokit.parameter_utils import get_fixed_gamma_beta

# Generating the circuits for benchmarking $N\cdot p$ of QAOA

One benchmark used for evaluating the ability of quantum computers to solve optimization problems is the maximum $N\cdot p$ that can be executed on this quantum computer. This benchmark was originally proposed by DARPA in 2019 [1] and has been demonstrated on Quantinuum H1 [2].

In this notebook, we show how to generate the circuits for this evaluation using QOKit.

[1] Optimization with Noisy Intermediate-Scale Quantum devices (ONISQ). Notice ID: HR001119S0052 https://sam.gov/opp/1e94e435a6be3b2f0939c8eeea1d7fd3/view 

[2] QAOA with $N\cdot p\geq 200$. Ruslan Shaydulin, Marco Pistoia https://arxiv.org/abs/2303.02064

In [2]:
# Generate a random regular graph
N = 10
d = 3
seed = 1
G = nx.random_regular_graph(d,N,seed=seed)

p = 11
# Get the parameters for QAOA for MaxCut on regular graphs from arXiv:2107.00677
gamma, beta = get_fixed_gamma_beta(d, p)

# Get circuit
qc = get_qaoa_circuit(G, gamma, beta, save_statevector=False)
qc.measure_all()

## Optional: execute and validate the solution quality

In [3]:
# Compute true optimal solution
obj = partial(maxcut_obj, w=get_adjacency_matrix(G))
optimal_cut = brute_force(obj, N, function_takes="bits")[0]

In [4]:
from qiskit.providers.aer import AerSimulator

backend_qiskit = AerSimulator()

In [5]:
counts_qiskit = invert_counts(backend_qiskit.run(qc).result().get_counts())

In [6]:
print(f"Approximation ratio: {objective_from_counts(counts_qiskit, obj) / optimal_cut}")

Approximation ratio: 0.96728515625


## Optional: try optimizing the parameters further

We will use the fast QOKit simulator instead of Qiskit for the optimization

In [7]:
from scipy.optimize import minimize

from qokit.qaoa_objective_maxcut import get_qaoa_maxcut_objective
from qokit.utils import precompute_energies

In [8]:
precomputed_energies = precompute_energies(obj, N)

In [9]:
# objective to optimize
f = get_qaoa_maxcut_objective(N, p, precomputed_cuts=precomputed_energies)
# putting minus in front of gamma since we flipped the sign for minimization
X0 = np.hstack([np.array(gamma), beta])

In [10]:
print(f"Approximation ratio before optimization: {-f(X0) / optimal_cut}")

Approximation ratio before optimization: 0.9675931950044419


In [11]:
res = minimize(f, X0, method='COBYLA', options={'rhobeg':0.001})

In [12]:
print(f"Approximation ratio after optimization: {-f(res.x) / optimal_cut}")

Approximation ratio after optimization: 0.9914728555522948


---